# Run This and Enter Date


In [34]:
import requests
import pandas as pd
import os
import openpyxl
# Use a session to maintain headers/cookies
session = requests.Session()
wb=openpyxl.Workbook()
commodity_data = {}
date = input("Enter the date (MM/DD/YYYY): ")

# Set headers to fully mimic a browser
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Accept": "application/json, text/javascript, */*; q=0.01",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.cmegroup.com/markets/agriculture/grains/wheat.settlements.html",
    "X-Requested-With": "XMLHttpRequest",
    "Connection": "keep-alive",
    "Origin": "https://www.cmegroup.com",
    "Host": "www.cmegroup.com"
})

product_codes = {"Corn":300,"Soybeans":320,"Soybean Meal":310}
product_dfs = {}



os.makedirs("Scraped_Data", exist_ok = True)

for product, code in product_codes.items():
# CME API endpoint and parameters
    date = date.replace("-","/")
    url = f"https://www.cmegroup.com/CmeWS/mvc/Settlements/Futures/Settlements/{code}/FUT"
    params = {
        "strategy": "DEFAULT",
        "tradeDate": date,
        "pageSize": 500,
        "isProtected": "",
        "_t": "1751224455214"  # Optional random number
    }

    # Send the request
    response = session.get(url, params=params)
    response.raise_for_status()  # This will raise error if still blocked

    # Parse the response
    data = response.json()
    df = pd.DataFrame(data["settlements"])
    df = df[["month", "settle",'change']]
    df.set_index("month",inplace=True)
    df.drop(index="Total")
    product_dfs[f"{product}"] = df
date = date.replace("/","-")
excel_path = f"Scraped_Data/{date}.xlsx"
with pd.ExcelWriter(excel_path,engine='openpyxl') as writer:
    for product ,df in product_dfs.items():
        df.to_excel(writer,sheet_name=f"{product}")

